In [ ]:
!pip3 install google-cloud-aiplatform --upgrade --user

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [ ]:
worker_pool_specs = [{
    "machine_spec": {
        "machine_type": "n1-standard-4",
        "accelerator_type": "NVIDIA_TESLA_T4",
        "accelerator_count": 1
    },
    "replica_count": 1,
    "container_spec": {
        "image_uri": "gcr.io/vertex-ai-123/rps:v1"
    }
}]


#Optimizational goal of the metric in dict format
metric_spec={'accuracy':'maximize'}

# Hyperparameters which needs to be optimized

parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "momentum": hpt.DoubleParameterSpec(min=0, max=1, scale="linear"),
    "num_neurons": hpt.DiscreteParameterSpec(values=[16, 32, 64], scale=None)
}


In [ ]:
my_custom_job = aiplatform.CustomJob(display_name='rps_classification-sdk-job',
                              worker_pool_specs=worker_pool_specs,
                              staging_bucket='gs://vertex-ai-custom')
#To run the custom jobs
#my_custom_job.run()

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name='rps_classification-sdk-job',
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=15,
    parallel_trial_count=3)
#To initiate the hyperparameyter tuning job
hp_job.run()